# INTELIGENCIA ARTIFICIAL
## Tarea 1: implementacion de gradiente descendiente
### Hecho por: Juan Sebastián Clavijo Martínez<br />
TEMA: Usar el dataset sobre finca raiz en taiwan para hacer una implementación de gradiente descendiente<br />
Fecha: 11-08-2024<br />
Notas: Nada por ahora<br />
**Pontificia Universidad Javeriana**

## 1. preprocesamiento de datos

In [77]:
# Importado de bibliotecas pertinentes
# se usa el #type: ignore para que mi vscode genere warnings sobre la resolucion de dependencias de los pkts

import pandas as pd  # tratamiento de datos en un dataframe
import numpy as np  # numerical python
import seaborn as sns  # biblioteca versatil para estadistica y visualizacion
import matplotlib.pyplot as plt  # biblioteca para vizualizacion de datos
import ipywidgets as widgets  # widgets interactivos, barras deslizantes, etc, para mas info mirar: https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#file-upload
from ipywidgets import widgets, interactive, VBox
from IPython.display import display # display interactivo
from ucimlrepo import fetch_ucirepo  # se traen los datos del repositorio, para mas info revisar: https://github.com/uci-ml-repo/ucimlrepo (el dataset tiene el id=447)

In [78]:
# Parte 1.1: carga de datos
# se trae el dataset
url = 'https://archive.ics.uci.edu/static/public/477/data.csv'
# se hace el obj dataframe
df_rev = pd.read_csv(url) ##Se llama así por Real Estate Valuation
# se presentan los primeros 5 renglones y la forma de los datos
df_rev.head(5)

,No,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
0,1,2012.917,32.0,84.87882,10,24.98298,121.54024,37.9
1,2,2012.917,19.5,306.59470,9,24.98034,121.53951,42.2
2,3,2013.583,13.3,561.98450,5,24.98746,121.54391,47.3
3,4,2013.500,13.3,561.98450,5,24.98746,121.54391,54.8
4,5,2012.833,5.0,390.56840,5,24.97937,121.54245,43.1


In [79]:
# Parte 1.2: limpieza y preparación de los datos (cuenta de datos null, etc. y limpieza)
desaparecidos = len(df_rev) - len(df_rev.dropna())
Cantidad = len(df_rev)
print("Cantidad de datos observados con datos NaN", desaparecidos)
print("Cantidad de datos duplicados", df_rev.duplicated().sum())
##se eliminan los datos Null y Duplicados
df_rev.dropna(inplace=True)
df_rev.drop_duplicates(inplace=True)
# Se reinicia el indice por los elementos eliminados
df_rev.reset_index(drop=True, inplace=True)

Cantidad de datos observados con datos NaN 0
Cantidad de datos duplicados 0


## 2. Implementación de Gradiente Descendiente

### Flujo de GD
1. proponer el modelo lineal (la matriz A refleja la estruc del modelo)
2. definir un theta inicial aleatoriamente
3. estimar salida del modelo para ese theta h=A*theta
4. encontrar el error
5. actualizar el theta = theta - (alpha/m(numero de datos))xA^t*e
6. $y=mx+b$
7. goto 3 until convergence
    - crieterios de parada:
      - ubmral de error
      - no hay cambio en theta
- util: no actualizar h! solo se vuelve a calcular cuando se actualizan los parametros

In [80]:
# Parte 2.2 Implementación de GD

# Lista para almacenar los valores de theta y error en cada iteración
theta_values = []
error_values = []

# Implementación de las funciones necesarias para GD


def calculo_h(theta, mat_A):
    return mat_A @ theta


def calculo_error(h, y):
    return h - y


def calc_theta(mat_A, y):
    return np.linalg.pinv(mat_A.T @ mat_A) @ mat_A.T @ y


def calc_h0(theta, mat_A):
    return np.matmul(mat_A, theta)


def calc_cost_J_m(theta, h_theta, y):
    e = y - h_theta
    m = y.shape[0]
    J = e.T @ e * (1 / m)
    return J.item()  # Asegurarse de que J sea un valor escalar


def calc_gradient_J_m(mat_A, h_theta, y):
    m = y.shape[0]
    e = y - h_theta
    gradient_J = (1 / m) * (-2 * np.dot(np.transpose(mat_A), e))
    return gradient_J


def update_theta_params(alfa, mat_A, h_theta, y, theta):
    new_theta = theta - alfa * calc_gradient_J_m(mat_A, h_theta, y)
    return new_theta.reshape(-1, 1)


# Implementación del gradiente descendente

df_rev_01 = df_rev.rename(
    columns={
        "X1 transaction date": "transaction date",
        "X2 house age": "house age",
        "X3 distance to the nearest MRT station": "distance to the nearest MRT station",
        "X4 number of convenience stores": "number of convenience stores",
        "X5 latitude": "latitude",
        "X6 longitude": "longitude",
        "Y house price of unit area": "house price of unit area",  # Renombrar la columna Y también
    }
)
x_columns = df_rev_01.columns.drop(["house price of unit area", "No"])


def gradiente_descendiente(iteraciones, alpha):
    theta = np.array([np.random.normal(1, 10), np.random.normal(1, 10)]).reshape(
        -1, 1
    )  # Ajustado para dos parámetros
    y = np.array(df_rev_01["house price of unit area"]).reshape(-1, 1)

    x_1 = np.array(df_rev_01["house age"]).reshape(-1, 1)
    A = np.column_stack(
        (x_1, np.ones((len(df_rev_01), 1)))
    )  # Ajustado para dos parámetros

    theta_saved = []
    cost = []

    for i in range(iteraciones):
        h_theta = calc_h0(theta, A)
        J = calc_cost_J_m(theta, h_theta, y)
        cost.append(J)
        theta = update_theta_params(alpha, A, h_theta, y, theta)
        theta_saved.append(list(theta))

    return theta_saved, cost


# Función para la gráfica de la derecha (scatterplot interactivo)
def plot_graph(x_column):
    x = df_rev_01[x_column]
    y = df_rev_01["house price of unit area"]

    plt.figure(figsize=(6.5, 6.5))
    plt.scatter(x, y, alpha=0.3, edgecolor="none")
    plt.xlabel(x_column)
    plt.ylabel("house price of unit area [10k TWD/Ping (1 Ping = 3.3 m²)]")
    plt.title("Juega con el eje x!")


# Función para la gráfica de gradiente descendente en 3D
def plot_gd_graph(iteraciones, alpha):
    theta_values, cost = gradiente_descendiente(iteraciones, alpha)

    # Extraer los valores de theta
    theta_0 = [theta[0][0] for theta in theta_values]
    theta_1 = [theta[1][0] for theta in theta_values]

    # Crear la figura y el conjunto de ejes 3D
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection="3d")

    # Graficar los puntos en 3D
    img = ax.scatter(theta_0, theta_1, cost, c=cost, cmap=plt.gray(), marker="o")

    # Añadir barra de color
    cbar = fig.colorbar(img)
    cbar.set_label("Costo")  # Establecer la etiqueta de la barra de color

    # Configurar etiquetas y título
    ax.set_xlabel("Theta 0")
    ax.set_ylabel("Theta 1")
    ax.set_title("Relación del Costo con los Parámetros Theta")


# Sliders para iteraciones y tasa de aprendizaje
iter_slider = widgets.IntSlider(
    value=1000, min=100, max=5000, step=100, description="Iteraciones:"
)

alpha_slider = widgets.FloatSlider(
    value=0.001,
    min=0.001,
    max=0.1,
    step=0.001,
    description="Alpha:",
    readout=True,
    readout_format=".3f",
)

# Menú desplegable para la gráfica de la derecha
x_dropdown = widgets.Dropdown(
    options=x_columns, value="house age", description="Eje X:"
)


# Función para actualizar ambas gráficas
def update_plot(x_column, iteraciones, alpha):
    # First subplot: Scatter plot
    plot_graph(x_column)

    # Second subplot: Cost vs. theta plot
    plot_gd_graph(iteraciones, alpha)


# Widgets interactivos combinados
interactive_plot = interactive(
    update_plot, x_column=x_dropdown, iteraciones=iter_slider, alpha=alpha_slider
)
display(interactive_plot)

interactive(children=(Dropdown(description='Eje X:', index=1, options=('transaction date', 'house age', 'dista…